<a href="https://colab.research.google.com/github/LC1332/Speaker-Grouping/blob/main/notebook/%E5%B1%82%E6%AC%A1%E8%81%9A%E7%B1%BBbaseline%E6%B5%8B%E8%AF%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 载入parquet
- [ ] 进行层次聚类
- [ ] 进行基础的测评

# 载入Parquet，生成有待聚类的表格

In [1]:
import pandas as pd

# 读取 Parquet 文件到 pandas DataFrame
# data = pd.read_parquet("/content/liangjian_10_feature.parquet")
data = pd.read_parquet("/content/haruhi_03_feature.parquet")

# 打印所有列名
print("所有列名：")
for col in data.columns:
    print(col)


所有列名：
人物
人物台词
开始时间
结束时间
audio_file
screeshot_file
visual_feature
audio_feature


data是一个pandas的表格， 我希望生成一个新的表格 inference_data

inference_data相当于从data中去掉 人物 为none的所有行

然后进一步把人物这一列从inference_data中分离出来成为一个list叫做speakers

请用python为我实现

In [6]:
# 从data中删除人物为None的所有行
inference_data = data.dropna(subset=['人物'])

# 将人物这一列分离出来成为一个list
speakers = inference_data['人物'].tolist()

inference_data = inference_data.drop(columns=['人物'])

# 打印所有列名
print("所有列名：")
for col in inference_data.columns:
    print(col)

print(len(inference_data))
print(speakers[:10])

所有列名：
人物台词
开始时间
结束时间
audio_file
screeshot_file
visual_feature
audio_feature
440
['长门', '长门', '阿虚', '长门', '长门', '长门', '长门', '长门', '长门', '长门']


# audio特征形成baseline

我有一个pandas表格inference_data

我希望用表格中的audio_feature 进行cosine度量的 层次聚类，最终输出的类个数大约为10个，cosine similarity的阈值大约为0.55

请为我实现一个python函数，输入是inference_data ，输出是list of int为聚类的结果

In [27]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, fcluster

def hierarchical_clustering(data, sim_threshold=0.55, feature_name = "audio_feature"):
    """
    Perform hierarchical clustering on audio features using cosine similarity.

    :param data: DataFrame containing an 'audio_feature' column with audio features.
    :param num_clusters: Desired number of clusters, defaults to 10.
    :param sim_threshold: Cosine similarity threshold, defaults to 0.55.
    :return: List of integers representing cluster assignments.
    """
    # Extract audio features
    features = np.stack(data[feature_name].values)

    # Compute pairwise distances using cosine metric
    distances = pdist(features, metric='cosine')

    # Perform hierarchical clustering
    Z = linkage(distances, method='average')

    # Form clusters based on similarity threshold
    # Convert cosine similarity to distance
    distance_threshold = 1 - sim_threshold
    cluster_labels = fcluster(Z, t=distance_threshold, criterion='distance')

    # Create a new mapping for labels to make them continuous from 0
    unique_labels = np.unique(cluster_labels)
    label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
    continuous_labels = [label_mapping[label] for label in cluster_labels]

    return continuous_labels

# Since I cannot run this without the actual data, please run this on your local machine where your data is available.
# Example usage:
cluster_labels = hierarchical_clustering(inference_data,0.3, "audio_feature")
print(cluster_labels[:30])


[6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


# 进行统计

我现在有了聚类的结果

cluster_labels是一个list of int

以及speakers是一个list of string

我希望统计pairwise级别的聚类precision和recall

具体的步骤如下

1. 累计True Positive的边的个数

对于cluster_label = i 的所有结果，统计里面的不同speaker的样本个数

N_1, N_2, .... N_M

对TP边叔累计 sum_m N_m (N_m-1) /2

2. 计算 True Positive + False Postive的个数

对于各种cluster_label = i， 的样本数，如果是G_i

统计 sum_i G_i (G_i-1) /2

3. 计算 True Positive + False Negative的个数

对于每一种speaker，假定这个spearker共有 D_i 个

统计 sum_i D_i (D_i-1) / 2

统计完三者之后，计算Edge级别的Precision和Recall

实现一个python程序，输入是cluster_labels, speakers

输出一个dict，至少包含precision和recall字段

In [32]:
from collections import defaultdict
import itertools

def calculate_pairwise_precision_recall(cluster_labels, speakers):
    """
    Calculate pairwise precision and recall for clustering results based on speaker labels.

    :param cluster_labels: List of integers, cluster assignments for each sample.
    :param speakers: List of strings, speaker identifiers for each sample.
    :return: Dictionary containing 'precision' and 'recall'.
    """
    # Group indices by cluster and by speaker
    clusters = defaultdict(list)
    speaker_groups = defaultdict(list)

    for index, (cluster, speaker) in enumerate(zip(cluster_labels, speakers)):
        clusters[cluster].append(speaker)
        speaker_groups[speaker].append(index)

    # Calculate True Positives (TP): sum of intra-cluster possible pairs of same speaker
    TP = 0
    for cluster, members in clusters.items():
        speaker_count = defaultdict(int)
        for speaker in members:
            speaker_count[speaker] += 1
        print(speaker_count)
        for count in speaker_count.values():
            TP += count * (count - 1) / 2

    # Calculate True Positives + False Positives (TP + FP): sum of intra-cluster possible pairs
    TP_FP = 0
    for cluster, members in clusters.items():
        count = len(members)
        TP_FP += count * (count - 1) / 2

    # Calculate True Positives + False Negatives (TP + FN): sum of all possible pairs of same speaker across clusters
    TP_FN = 0
    for speaker, indices in speaker_groups.items():
        count = len(indices)
        TP_FN += count * (count - 1) / 2

    # Compute precision and recall
    precision = TP / TP_FP if TP_FP > 0 else 0
    recall = TP / TP_FN if TP_FN > 0 else 0

    return {'precision': precision, 'recall': recall, 'TP': TP, 'TP_FP':TP_FP, 'TP_FN':TP_FN}

# Example usage:
results = calculate_pairwise_precision_recall(cluster_labels, speakers)
print(results)



defaultdict(<class 'int'>, {'长门': 60})
defaultdict(<class 'int'>, {'阿虚': 5})
defaultdict(<class 'int'>, {'阿虚': 149})
defaultdict(<class 'int'>, {'阿虚': 4})
defaultdict(<class 'int'>, {'春日': 67, '朝比奈': 2})
defaultdict(<class 'int'>, {'春日': 5, '朝比奈': 6})
defaultdict(<class 'int'>, {'朝比奈': 56, '阿虚': 1, '春日': 1})
defaultdict(<class 'int'>, {'朝比奈': 1})
defaultdict(<class 'int'>, {'古泉': 1})
defaultdict(<class 'int'>, {'古泉': 75})
defaultdict(<class 'int'>, {'春日': 2})
defaultdict(<class 'int'>, {'阿虚': 2})
defaultdict(<class 'int'>, {'阿虚': 1})
defaultdict(<class 'int'>, {'春日': 1})
defaultdict(<class 'int'>, {'阿虚': 1})
{'precision': 0.9858982843761136, 'recall': 0.8511405089438755, 'TP': 19366.0, 'TP_FP': 19643.0, 'TP_FN': 22753.0}


In [31]:
print(30*59)

1770


# 测一下亮剑

In [33]:
import pandas as pd

# 读取 Parquet 文件到 pandas DataFrame
data = pd.read_parquet("/content/liangjian_10_feature.parquet")

# 从data中删除人物为None的所有行
inference_data = data.dropna(subset=['人物'])

# 将人物这一列分离出来成为一个list
speakers = inference_data['人物'].tolist()

inference_data = inference_data.drop(columns=['人物'])

cluster_labels = hierarchical_clustering(inference_data,0.3, "audio_feature")
print(cluster_labels[:30])

# Example usage:
results = calculate_pairwise_precision_recall(cluster_labels, speakers)
print(results)

[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 37, 23, 23, 23, 18, 23, 23, 14, 37, 31, 52, 52, 8, 61, 46, 25, 28, 28, 46, 46]
defaultdict(<class 'int'>, {'段鹏': 10})
defaultdict(<class 'int'>, {'李云龙': 157})
defaultdict(<class 'int'>, {'李云龙': 5, '方立功': 1, '楚云飞': 1})
defaultdict(<class 'int'>, {'李云龙': 2, '段鹏': 1, '和尚': 1})
defaultdict(<class 'int'>, {'李云龙': 2})
defaultdict(<class 'int'>, {'李云龙': 1})
defaultdict(<class 'int'>, {'和尚': 2})
defaultdict(<class 'int'>, {'和尚': 1})
defaultdict(<class 'int'>, {'卧底': 1, '段鹏': 1, '小二': 2, '李云龙': 4, '方立功': 1, '和尚': 1, '楚云飞': 1, '路人士兵': 2, '站岗士兵': 1})
defaultdict(<class 'int'>, {'和尚': 26, '李云龙': 1})
defaultdict(<class 'int'>, {'卧底': 1, '楚云飞': 132, '李云龙': 1, '赵刚': 1, '方立功': 1})
defaultdict(<class 'int'>, {'卧底': 23, '和尚': 2})
defaultdict(<class 'int'>, {'店员': 1, '秀芹': 17})
defaultdict(<class 'int'>, {'李云龙': 9})
defaultdict(<class 'int'>, {'段鹏': 29, '赵刚': 1, '李云龙': 1})
defaultdict(<class 'int'>, {'和尚': 1})
defaultdict(<class 'int'>, {'段鹏': 1})
defaultdict(<cla